.ipynb的介面最上面有一排東西"+Code", "+Markdown", ...最後有個"Outline"打開就可以看到目錄！

## 預處理

In [ ]:
import os  # 處理檔案路徑
import numpy as np  # 處理數值資料
import pandas as pd  # 處理表格型資料
from tqdm import tqdm  # 讓迴圈加上進度條
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from decimal import Decimal, ROUND_HALF_UP
from sklearn.metrics import accuracy_score




#有看到


In [ ]:
DATA_DIR = "/content/drive/MyDrive/39_Training_Dataset/39_Training_Dataset/train_data" # 存放 .txt 的資料夾
INFO_PATH = "/content/drive/MyDrive/39_Training_Dataset/39_Training_Dataset/train_info.csv" # 標籤的 .csv 檔路徑

In [ ]:
# 前處理：切割每筆資料的 27 次揮拍並提取統計特徵
from scipy.stats import kurtosis, skew
from scipy.signal import welch
from scipy.stats import entropy

def segment_swing(data, cut_points):
    return [data[cut_points[i]:cut_points[i+1]] for i in range(27)]

def extract_features(segment):
    feats = []

    acc = segment[:, 0:3]  # Ax, Ay, Az
    gyro = segment[:, 3:6] # Gx, Gy, Gz

    # 三軸平均值（共6）
    feats += list(acc.mean(axis=0))  # ax_mean, ay_mean, az_mean
    feats += list(gyro.mean(axis=0))  # gx_mean, gy_mean, gz_mean

    # 三軸變異數（共6）
    feats += list(acc.var(axis=0))  # ax_var, ...
    feats += list(gyro.var(axis=0))

    # 三軸 RMS（平方根平均）（共6）
    feats += list(np.sqrt((acc ** 2).mean(axis=0)))
    feats += list(np.sqrt((gyro ** 2).mean(axis=0)))

    # 加速度總量 & 角速度總量（不考慮方向）
    acc_total = np.linalg.norm(acc, axis=1)
    gyro_total = np.linalg.norm(gyro, axis=1)

    feats += [acc_total.max(), acc_total.min(), acc_total.mean()]  # 3
    feats += [gyro_total.max(), gyro_total.min(), gyro_total.mean()]  # 3

    # skewness + kurtosis + spectral entropy（各軸加總，共 6）
    for i in range(3):
        a = acc[:, i]
        g = gyro[:, i]

        # Skewness & Kurtosis
        feats += [skew(a), kurtosis(a)]
        feats += [skew(g), kurtosis(g)]

        # Spectral entropy (用 Welch 頻譜估計)
        for signal in [a, g]:
            f, Pxx = welch(signal, nperseg=min(len(signal), 64))
            Pxx /= Pxx.sum() + 1e-8
            feats += [entropy(Pxx)]

    return feats

def preprocess_dataset(info_csv_path, data_dir, is_train=True):
    df_info = pd.read_csv(info_csv_path)
    feature_list = []
    meta_list = []

    for idx, row in df_info.iterrows():
        uid = row["unique_id"]
        txt_path = os.path.join(data_dir, f"{uid}.txt")
        if not os.path.exists(txt_path):
            continue
        try:
            data = np.loadtxt(txt_path)
            cut_str = row["cut_point"]
            cut_points = list(map(int, cut_str.strip("[]").split()))
            if len(cut_points) != 28:
                continue  # 要切出27段，需28個點
            swings = segment_swing(data, cut_points)
            # 原有的特徵擷取
            all_feats = [extract_features(s) for s in swings]
            flatten_feats = np.concatenate(all_feats)  # shape: (1134,)

            # 🔁 新增 mode 特徵（10 維 one-hot）
            mode_idx = int(row["mode"]) - 1  # 從 1~10 → 0~9
            mode_onehot = np.zeros(10)
            if 0 <= mode_idx < 10:
                mode_onehot[mode_idx] = 1

            # ➕ 合併進整體特徵
            full_feat = np.concatenate([flatten_feats, mode_onehot])  # shape: (1144,)

            # ➕ 加入清單
            feature_list.append(full_feat)

            if is_train:
                meta_list.append({
                    "unique_id": uid,
                    "gender": 1 if row["gender"] == 2 else 0,
                    "hold racket handed": 1 if row["hold racket handed"] == 2 else 0,
                    "play years": row["play years"],
                    "level": row["level"] - 2  # 轉成 0~3
                })
            else:
                meta_list.append({"unique_id": uid})
        except:
            continue  # 忽略格式錯誤或缺失

    df_feat = pd.DataFrame(feature_list, columns=[f"f{i}" for i in range(len(feature_list[0]))])
    df_meta = pd.DataFrame(meta_list)
    df_result = pd.concat([df_meta, df_feat], axis=1)
    return df_result

# ⏬ 執行前處理
train_df = preprocess_dataset(INFO_PATH, DATA_DIR, is_train=True)
test_df = preprocess_dataset("/content/drive/MyDrive/39_Test_Dataset/39_Test_Dataset/test_info.csv","/content/drive/MyDrive/39_Test_Dataset/39_Test_Dataset/test_data", is_train=False)

train_df.to_csv("train_features.csv", index=False)
test_df.to_csv("test_features.csv", index=False)
print("✅ train_features.csv和test_features.csv 已完成前處理")


In [ ]:
print(train_df)

      unique_id  gender  hold racket handed  play years  level           f0  \
0             1       0                   0           1      3  3797.098361   
1             2       0                   0           1      3  1956.189189   
2             3       0                   0           1      3  1892.689320   
3             4       0                   0           1      3  1960.603960   
4             5       0                   0           1      3  1965.714286   
...         ...     ...                 ...         ...    ...          ...   
1950       1963       0                   0           0      2 -2235.329268   
1951       1964       0                   0           0      2 -3345.952941   
1952       1965       0                   0           0      2 -3284.554217   
1953       1966       0                   0           0      2 -3066.012346   
1954       1967       0                   0           0      2 -2877.564103   

               f1           f2            f3       

In [ ]:
print(test_df)

      unique_id           f0           f1           f2           f3  \
0          1968  1394.294737  1149.168421  -691.821053 -2895.084211   
1          1969  3150.474747  -982.111111  -496.494949  -364.474747   
2          1970  3945.687500 -1513.137500   146.212500  -842.912500   
3          1971  3232.700000 -2123.400000  -573.957143    52.771429   
4          1972  3976.363636 -1927.500000 -1202.227273  -356.409091   
...         ...          ...          ...          ...          ...   
1425       3407  3301.483516 -2525.890110   284.362637   589.208791   
1426       3408  2678.644444 -1519.500000  -750.533333 -1111.344444   
1427       3409  2989.430108  -344.268817  -628.301075 -4936.344086   
1428       3410  4225.342857 -3032.514286 -1703.400000  -642.266667   
1429       3411  3506.988764 -3023.134831  -860.595506  -111.011236   

               f4           f5            f6            f7            f8  ...  \
0    -4766.031579  3710.505263  2.211002e+04  2.145752e+05  3.6269

In [3]:
# 讀取前處理好的特徵資料（每筆 918 維 + 標籤）
info_df = pd.read_csv("train_features.csv")

# 分離特徵與標籤
X_data = info_df.drop(columns=["unique_id", "gender", "hold racket handed", "play years", "level"]).values

# 提取各任務的標籤
y_gender = info_df["gender"].values
y_hand   = info_df["hold racket handed"].values
y_years  = info_df["play years"].values
y_level  = info_df["level"].values

In [4]:
print("👨 男生有幾筆：", np.sum(y_gender == 0))
print("👩 女生有幾筆：", np.sum(y_gender == 1))
print("🫱 右手持拍有幾筆：", np.sum(y_hand == 0))
print("🫲 左手持拍有幾筆：", np.sum(y_hand == 1))
# 幾乎 5 倍差距 嚴重失衡
# 建議使用 oversampling / SMOTE 等技巧

# W1 使用 class_weight（推薦！）
# W2 上採樣少數類別（如女生）
# W3 設計多目標損失時，對 gender 特別調整權重

# 檢查持拍手分布
# 4.34 倍差距→ 明顯不平衡

# 檢查球齡分布
print("低球齡有幾筆：", np.sum(y_years == 0))
print("中球齡有幾筆：", np.sum(y_years == 1))
print("高球齡有幾筆：", np.sum(y_years == 2))
# 差 ≈ 2.2 倍 → 輕度不平衡

# 檢查等級分布
print("等級0：", np.sum(y_level == 0))
print("等級1：", np.sum(y_level == 1))
print("等級2：", np.sum(y_level == 2))
print("等級3：", np.sum(y_level == 3))
# 6.6 倍差距 → 明顯不平衡

👨 男生有幾筆： 1627
👩 女生有幾筆： 328
🫱 右手持拍有幾筆： 1589
🫲 左手持拍有幾筆： 366
低球齡有幾筆： 387
中球齡有幾筆： 868
高球齡有幾筆： 700
等級0： 715
等級1： 201
等級2： 136
等級3： 903


In [5]:
# 存下來，之後大家可以直接用
# 如果你資料預處理有更動的話，下面的檔名也記得改！！！
# 更改方案，這些檔案就自己可以存下來，不要上傳到github上
np.save("/content/X_data.npy", X_data)
np.save("/content/y_gender.npy", y_gender)
np.save("/content/y_hand.npy", y_hand)
np.save("/content/y_years.npy", y_years)
np.save("/content/y_level.npy", y_level)

## Test
看一下資料分布而已，也可以跳過

## 0. 載資料

In [65]:
# 切分訓練與驗證資料（80% 訓練, 20% 驗證）
X_train, X_val, y_gender_train, y_gender_val, y_hand_train, y_hand_val, y_years_train, y_years_val, y_level_train, y_level_val = train_test_split(
    X_data, y_gender, y_hand, y_years, y_level, test_size=0.2, random_state=42
)

## Optional. 權重處理

In [66]:
# 處理球齡資料不平衡


weights_years = compute_class_weight(class_weight='balanced', classes=np.unique(y_years), y=y_years)
weights_gender = compute_class_weight(class_weight='balanced', classes=np.unique(y_gender), y=y_gender)
weights_hands = compute_class_weight(class_weight='balanced', classes=np.unique(y_hand), y=y_hand)
weights_level = compute_class_weight(class_weight='balanced', classes=np.unique(y_level), y=y_level)
print("class weights_years:", dict(enumerate(weights_years)))
print("class weights_gender:", dict(enumerate(weights_gender)))
print("class weights_hands:", dict(enumerate(weights_hands)))
print("class weights_level:", dict(enumerate(weights_level)))

pos_weight_gender = torch.tensor([weights_gender[1]], dtype=torch.float32)
pos_weight_hand   = torch.tensor([weights_hands[1]], dtype=torch.float32)
ce_weight_years   = torch.tensor(weights_years, dtype=torch.float32)
ce_weight_level   = torch.tensor(weights_level, dtype=torch.float32)

print("pos_weight_gender =", pos_weight_gender.item())
print("pos_weight_hand =", pos_weight_hand.item())

class weights_years: {0: np.float64(1.6838931955211025), 1: np.float64(0.750768049155146), 2: np.float64(0.930952380952381)}
class weights_gender: {0: np.float64(0.60079901659496), 1: np.float64(2.980182926829268)}
class weights_hands: {0: np.float64(0.6151667715544368), 1: np.float64(2.670765027322404)}
class weights_level: {0: np.float64(0.6835664335664335), 1: np.float64(2.431592039800995), 2: np.float64(3.59375), 3: np.float64(0.54125138427464)}
pos_weight_gender = 2.9801828861236572
pos_weight_hand = 2.670764923095703


## 1. 定義資料集

In [67]:
# Dataset
# 自定義一個能被 DataLoader 使用的資料類別
# 當 DataLoader 做 batch 時會呼叫這個函式

class TableTennisDataset(Dataset):
    def __init__(self, X, y_gender, y_hand, y_years, y_level):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y_gender = torch.tensor(y_gender, dtype=torch.float32)
        self.y_hand = torch.tensor(y_hand, dtype=torch.float32)
        self.y_years = torch.tensor(y_years, dtype=torch.long)
        self.y_level = torch.tensor(y_level, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], {
            'gender': self.y_gender[idx],
            'hand': self.y_hand[idx],
            'years': self.y_years[idx],
            'level': self.y_level[idx]
        }

## 2. 定義模型

In [68]:
# 定義損失函數
# 要不要加權
# use_class_weight = False
use_class_weight = True

if use_class_weight:
    # binary 分類：性別 & 手 → BCEWithLogitsLoss + pos_weight
    loss_gender = nn.BCEWithLogitsLoss(pos_weight=pos_weight_gender)
    loss_hand   = nn.BCEWithLogitsLoss(pos_weight=pos_weight_hand)

    # 多類別分類：球齡 & 等級 → CrossEntropyLoss + weight
    loss_years = nn.CrossEntropyLoss(weight=ce_weight_years)
    loss_level = nn.CrossEntropyLoss(weight=ce_weight_level)
else:
    loss_gender = nn.BCEWithLogitsLoss()
    loss_hand   = nn.BCEWithLogitsLoss()
    loss_years  = nn.CrossEntropyLoss()
    loss_level  = nn.CrossEntropyLoss()

# def compute_loss(preds, targets):
#     loss_gender = loss_gender(preds['gender'], targets['gender'])
#     loss_hand = loss_hand(preds['hand'], targets['hand'])
#     loss_years = loss_years(preds['years'], targets['years'])
#     loss_level = loss_level(preds['level'], targets['level'])

#     total_loss = loss_gender + loss_hand + loss_years + loss_level
#     return total_loss

In [69]:
from collections import deque

class TaskLossTracker:
    def __init__(self, max_len=100):
        self.history = {
            'gender': deque(maxlen=max_len),
            'hand': deque(maxlen=max_len),
            'years': deque(maxlen=max_len),
            'level': deque(maxlen=max_len),
        }

    def update(self, loss_g, loss_h, loss_y, loss_l):
        self.history['gender'].append(loss_g.item())
        self.history['hand'].append(loss_h.item())
        self.history['years'].append(loss_y.item())
        self.history['level'].append(loss_l.item())

    def get_weights(self):
        # 若某項為空，給一個預設值避免除以 0
        avg = {
            k: sum(v) / len(v) if len(v) > 0 else 1.0
            for k, v in self.history.items()
        }

        inv = {k: 1.0 / v for k, v in avg.items()}
        total_inv = sum(inv.values())
        weights = {k: v / total_inv for k, v in inv.items()}
        return weights


In [74]:
import torch.nn as nn

class TableTennisMLP(nn.Module):
    def __init__(self, input_size=1144, hidden_size=256):
        super().__init__()
        self.shared = nn.Sequential(
          nn.Linear(input_size, 512),
          nn.BatchNorm1d(512),
          nn.ReLU(),
          nn.Dropout(0.2),
          nn.Linear(512, 256),
          nn.BatchNorm1d(256),
          nn.ReLU(),
          nn.Dropout(0.2)
      )

        # 多輸出 heads
        self.fc_gender = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )  # sigmoid
        self.fc_hand = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.fc_years  = nn.Linear(hidden_size, 3)  # softmax
        self.fc_level  = nn.Linear(hidden_size, 4)  # softmax

    def forward(self, x):  # x shape: (batch, 918)
        x = self.shared(x)

        gender = self.fc_gender(x).view(-1)#gender = torch.sigmoid(self.fc_gender(x)).squeeze(1)
        hand   = self.fc_hand(x).view(-1)# hand   = torch.sigmoid(self.fc_hand(x)).squeeze(1)
        years  = self.fc_years(x)
        level  = self.fc_level(x)

        return {
            "gender": gender,
            "hand": hand,
            "years": years,
            "level": level
        }


## 3. 定義模型訓練

In [75]:
# train
def train(model, train_loader, optimizer, device):
      model.train()
      total_loss = 0
      correct_gender, correct_hand, correct_years, correct_level = 0, 0, 0, 0
      total_samples = 0

      for x, y in tqdm(train_loader):
            x = x.to(device)
            y_gender = y['gender'].float().to(device)  # ✅ for BCEWithLogitsLoss
            y_hand = y['hand'].float().to(device)
            y_years = y['years'].to(device)
            y_level = y['level'].to(device)
            # y = {k: v.to(device) for k, v in y.items()}

            optimizer.zero_grad()
            # 初始化追蹤器
            loss_tracker = TaskLossTracker(max_len=100)
            out = model(x)


            # 個別損失
            loss_g = loss_gender(out['gender'], y_gender)
            loss_h = loss_hand(out['hand'], y_hand)
            loss_y = loss_years(out['years'], y_years)
            loss_l = loss_level(out['level'], y_level)
            # loss = loss_g + loss_h + loss_y + loss_l
            # 更新 tracker
            loss_tracker.update(loss_g, loss_h, loss_y, loss_l)

            # 動態權重
            weights = loss_tracker.get_weights()

            # 加權總 loss
            loss = (
                  80 * weights['gender'] * loss_g +
                  80 * weights['hand']  * loss_h +
                  weights['years']  * loss_y +
                  weights['level']  * loss_l
            )
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # ➕ 準確率計算
            total_samples += x.size(0)

            pred_gender = (torch.sigmoid(out['gender']) > 0.5).float()
            correct_gender += (pred_gender == y_gender).sum().item()

            pred_hand = (torch.sigmoid(out['hand']) > 0.5).float()
            correct_hand += (pred_hand == y_hand).sum().item()

            pred_years = torch.argmax(out['years'], dim=1)
            correct_years += (pred_years == y_years).sum().item()

            pred_level = torch.argmax(out['level'], dim=1)
            correct_level += (pred_level == y_level).sum().item()

            # 訓練結果統計
            acc_gender = correct_gender / total_samples
            acc_hand = correct_hand / total_samples
            acc_years = correct_years / total_samples
            acc_level = correct_level / total_samples
            avg_loss = total_loss / len(train_loader)

      print(f"Train Loss: {avg_loss:.4f} | Train Acc - "
            f"Gender: {acc_gender:.3f}, Hand: {acc_hand:.3f}, "
            f"Years: {acc_years:.3f}, Level: {acc_level:.3f}")
      print("out['gender'] shape:", out['gender'].shape)
      print("y_gender shape:", y_gender.shape)
      print("y_gender unique values:", y_gender.unique())
      print("pred_gender probs:", torch.sigmoid(out['gender']).mean().item())
      return avg_loss

## 4. 定義驗證函數

In [76]:
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    total_samples = 0
    correct = {'gender': 0, 'hand': 0, 'years': 0, 'level': 0}
    all_preds, all_labels = {k: [] for k in correct}, {k: [] for k in correct}

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = {
                "gender": y["gender"].float().to(device),
                "hand": y["hand"].float().to(device),
                "years": y["years"].to(device),
                "level": y["level"].to(device)
            }
            out = model(x)

            # loss（若你想統計）
            loss_g = loss_gender(out['gender'], y['gender'])
            loss_h = loss_hand(out['hand'], y['hand'])
            loss_y = loss_years(out['years'], y['years'])
            loss_l = loss_level(out['level'], y['level'])
            total_loss += (loss_g + loss_h + loss_y + loss_l).item()

            total_samples += x.size(0)

            # 預測 + 收集
            for key in ['gender', 'hand']:
                pred = (torch.sigmoid(out[key]) > 0.5).int()
                correct[key] += (pred == y[key].int()).sum().item()
                all_preds[key].extend(pred.cpu().numpy())
                all_labels[key].extend(y[key].int().cpu().numpy())

            for key in ['years', 'level']:
                pred = torch.argmax(out[key], dim=1)
                correct[key] += (pred == y[key]).sum().item()
                all_preds[key].extend(pred.cpu().numpy())
                all_labels[key].extend(y[key].cpu().numpy())

    accs = {k: correct[k] / total_samples for k in correct}
    avg_loss = total_loss / len(dataloader)

    print(f"\n[Validation] Loss: {avg_loss:.4f} | " +
          " | ".join([f"acc_{k}: {v:.3f}" for k, v in accs.items()]))

    return accs, all_preds, all_labels


## 主程式

In [77]:
# 使用 GPU 優先
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 載入模型
model = TableTennisMLP().to(device)
# 優化器
optimizer = optim.Adam(model.parameters(), lr=1e-4)

loss_gender.to(device)
loss_hand.to(device)
loss_years.to(device)
loss_level.to(device)
# 準備資料集與 DataLoader
train_dataset = TableTennisDataset(X_train, y_gender_train, y_hand_train, y_years_train, y_level_train)
val_dataset = TableTennisDataset(X_val, y_gender_val, y_hand_val, y_years_val, y_level_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

EPOCH = 100
for epoch in range(EPOCH + 1):
    print(f"\n🟢[Epoch {epoch}]")
    train_loss = train(model, train_loader, optimizer, device)
    val_loss = evaluate(model, val_loader, device)


🟢[Epoch 0]


100%|██████████| 49/49 [00:00<00:00, 207.83it/s]


Train Loss: 41.3568 | Train Acc - Gender: 0.708, Hand: 0.776, Years: 0.311, Level: 0.339
out['gender'] shape: torch.Size([28])
y_gender shape: torch.Size([28])
y_gender unique values: tensor([0., 1.], device='cuda:0')
pred_gender probs: 0.3794592618942261

[Validation] Loss: 4.0750 | acc_gender: 0.870 | acc_hand: 0.806 | acc_years: 0.458 | acc_level: 0.343

🟢[Epoch 1]


100%|██████████| 49/49 [00:00<00:00, 210.32it/s]


Train Loss: 37.8721 | Train Acc - Gender: 0.854, Hand: 0.811, Years: 0.431, Level: 0.379
out['gender'] shape: torch.Size([28])
y_gender shape: torch.Size([28])
y_gender unique values: tensor([0., 1.], device='cuda:0')
pred_gender probs: 0.33970609307289124

[Validation] Loss: 3.8203 | acc_gender: 0.880 | acc_hand: 0.798 | acc_years: 0.522 | acc_level: 0.391

🟢[Epoch 2]


...

Train Loss: 9.3216 | Train Acc - Gender: 0.971, Hand: 0.918, Years: 0.685, Level: 0.550
out['gender'] shape: torch.Size([28])
y_gender shape: torch.Size([28])
y_gender unique values: tensor([0., 1.], device='cuda:0')
pred_gender probs: 0.23266854882240295

[Validation] Loss: 3.7866 | acc_gender: 0.875 | acc_hand: 0.806 | acc_years: 0.637 | acc_level: 0.409

🟢[Epoch 99]


100%|██████████| 49/49 [00:00<00:00, 211.30it/s]


Train Loss: 9.2319 | Train Acc - Gender: 0.976, Hand: 0.943, Years: 0.663, Level: 0.551
out['gender'] shape: torch.Size([28])
y_gender shape: torch.Size([28])
y_gender unique values: tensor([0., 1.], device='cuda:0')
pred_gender probs: 0.09351038187742233

[Validation] Loss: 3.6246 | acc_gender: 0.890 | acc_hand: 0.816 | acc_years: 0.614 | acc_level: 0.412

🟢[Epoch 100]


100%|██████████| 49/49 [00:00<00:00, 175.57it/s]


Train Loss: 9.1050 | Train Acc - Gender: 0.980, Hand: 0.935, Years: 0.665, Level: 0.571
out['gender'] shape: torch.Size([28])
y_gender shape: torch.Size([28])
y_gender unique values: tensor([0., 1.], device='cuda:0')
pred_gender probs: 0.22650852799415588

[Validation] Loss: 3.7276 | acc_gender: 0.895 | acc_hand: 0.783 | acc_years: 0.629 | acc_level: 0.407


## 儲存模型

In [78]:
torch.save(model.state_dict(), 'modelWeight042400.pth')

## 測資

In [79]:
# 1. 讀取前處理後的測試特徵資料
def load_test_features(test_csv_path="test_features.csv"):
    df = pd.read_csv(test_csv_path)
    uids = df["unique_id"].tolist()
    X = df.drop(columns=["unique_id"]).values
    return uids, torch.tensor(X, dtype=torch.float32)

In [80]:
# 2. 預測函式（針對特徵輸入設計）
def predict_from_features(model, X_test_tensor, device):
    model.eval()
    preds = []

    with torch.no_grad():
        for x in tqdm(X_test_tensor):
            x = x.unsqueeze(0).to(device)  # 加 batch 維度
            out = model(x)

            gender_logit = out['gender']
            hand_logit = out['hand']
            years_logits = out['years']
            level_logits = out['level']

            gender = torch.sigmoid(gender_logit).item()
            hand = torch.sigmoid(hand_logit).item()

            years = torch.softmax(years_logits, dim=1).squeeze(0).cpu().numpy()
            level = torch.softmax(level_logits, dim=1).squeeze(0).cpu().numpy()

            preds.append([gender, hand] + years.tolist() + level.tolist())

    return preds

In [81]:
# 3. 建立提交檔案
from decimal import Decimal, ROUND_HALF_UP
def create_submission(uids, preds, save_path='submission.csv'):
    def format_float(val):
        return str(Decimal(val).quantize(Decimal('0.000001'), rounding=ROUND_HALF_UP))

    df = pd.DataFrame(preds, columns=[
        'gender', 'hold racket handed',
        'play years_0', 'play years_1', 'play years_2',
        'level_2', 'level_3', 'level_4', 'level_5'
    ])
    df.insert(0, 'unique_id', uids)

    # ✅ 在這裡加入 NaN 補值：用每欄平均值填補
    df.fillna(df.mean(numeric_only=True), inplace=True)

    # 格式化小數位
    for col in df.columns[1:]:
        df[col] = df[col].apply(format_float)

    # 儲存 CSV
    with open(save_path, "w", encoding="utf-8", newline='\n') as f:
        df.to_csv(f, index=False)
    print(f"✔️ Submission saved to {save_path}")


In [82]:
# 1. 載入 test data
TEST_FEATURES = "/content/test_features.csv"

uids, X_test_tensor = load_test_features(TEST_FEATURES)
model = TableTennisMLP().to(device)
model.load_state_dict(torch.load("/content/modelWeight042400.pth"))
model.eval()

# 2. 預測
preds = predict_from_features(model, X_test_tensor, device)

# 3. 產出 CSV
submissionCSV = "submission2.csv" # 記得改
create_submission(uids, preds, save_path=submissionCSV)


100%|██████████| 1430/1430 [00:01<00:00, 1071.67it/s]

✔️ Submission saved to submission2.csv


In [36]:
def generate_ground_truth_csv(filename='ground_truth.csv', num_samples=1430, seed=42):
    np.random.seed(seed)

    data = {
        'unique_id': [1000 + i for i in range(1, num_samples + 1)],
        'gender': np.random.randint(0, 2, size=num_samples),  # 0 or 1
        'hold racket handed': np.random.randint(0, 2, size=num_samples),  # 0 or 1
        'play years': np.random.randint(0, 3, size=num_samples),  # 0, 1, 2
        'level': np.random.randint(2, 6, size=num_samples),  # 2, 3, 4, 5
    }

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"✅ ground_truth.csv 已成功生成，共 {num_samples} 筆資料。")

# 執行生成測資
generate_ground_truth_csv()

✅ ground_truth.csv 已成功生成，共 1430 筆資料。


In [37]:
from sklearn.metrics import roc_auc_score


def evaluate_submission(submission_path, ground_truth_path):
    # 讀取資料
    pred = pd.read_csv(submission_path)
    true = pd.read_csv(ground_truth_path)

    # ⭐ 補上這行：將所有數值欄位中的 NaN 填補為該欄位平均
    pred.fillna(pred.mean(numeric_only=True), inplace=True)

    # 二元類別
    gender_auc = roc_auc_score(true['gender'], pred['gender'])
    hand_auc = roc_auc_score(true['hold racket handed'], pred['hold racket handed'])

    # 多類別 - One-vs-Rest (Macro-average)
    years_true = true['play years']
    years_pred = pred[['play years_0', 'play years_1', 'play years_2']]
    years_auc = roc_auc_score(years_true, years_pred, multi_class='ovr', average='macro')

    level_true = true['level']
    level_pred = pred[['level_2', 'level_3', 'level_4', 'level_5']]
    level_auc = roc_auc_score(level_true, level_pred, multi_class='ovr', average='macro')

    # 最終分數
    final_score = (gender_auc + hand_auc + years_auc + level_auc) / 4

    print(f" ROC AUC - Gender: {gender_auc:.4f}")
    print(f" ROC AUC - Hand:   {hand_auc:.4f}")
    print(f" ROC AUC - Years:  {years_auc:.4f}")
    print(f" ROC AUC - Level:  {level_auc:.4f}")
    print(f"\n Final Score:     {final_score:.4f}")

    return final_score

evaluate_submission("submission2.csv", "ground_truth.csv")


 ROC AUC - Gender: 0.4983
 ROC AUC - Hand:   0.4905
 ROC AUC - Years:  0.5038
 ROC AUC - Level:  0.5016

 Final Score:     0.4985


np.float64(0.49854402452278707)